# journalists scraper

## Load Libraries

In [1]:
# Selenium
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
# Reqeusts
import requests
# Other tools
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta
from datetime import date
import csv
import pandas as pd
import time
import json
import fnmatch
import os
import tabula
from tabula.io import read_pdf


## Functions

In [2]:
def requests_get_item(url, item):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
    if item == "html":       
        page = requests.get(url, headers = headers)
        soup = BeautifulSoup(page.text, 'html.parser')
        return(soup)
    
    elif item == "pdf":
        page = requests.get(url, headers = headers, stream=True)
        return(page)
    else:
        print("Valid Item Not Selected")

def download_pdfs(ls_pdf_urls, download_path, file_name):
    counter = 0 
    for pdf_url in ls_pdf_urls:
        counter = counter +1
        g = requests_get_item(pdf_url, "pdf")
        with open(f'{download_path}{file_name}_{counter}.pdf', 'wb') as sav:
            for chunk in g.iter_content(chunk_size=1000000):
                sav.write(chunk)
        print(f"download number: {counter}")
                
                
def convert_pdf_to_csv(pdf_directory, csv_directory):
    directory = fr'{pdf_directory}'
    directory_output = fr'{csv_directory}'
    count = 0
    for file in os.listdir(directory):        
        if file.endswith(".pdf"):
            count = count + 1 
            print(f'{directory}{file}: Conversion {count}')
            tabula.convert_into(f'{directory}{file}', f'{directory_output}{count}.csv', output_format="csv", pages='all')
    
    
    
    

## Functions Using Selenium

In [3]:
# Define important paths
executable_path = "/Users/aadittambe/Documents/UMD/code_practice/news_app/repo/chromedriver"
download_path = f"test"

# Define Routes (usually in xpaths)

def browser_instance(executable_path, headless, download_path):
    
    
    chrome_options =  Options()
    if headless:
        chrome_options.add_argument("--headless")   
    chrome_options.add_experimental_option("prefs", {
            "download.default_directory": download_path})    
    driver = webdriver.Chrome(executable_path=executable_path, options = chrome_options)
    return driver  

def signIn(driver, url, signin_field, username_field, password_field):  
    driver.get(url)
    sign_in = driver.find_element_by_xpath(signin_field)
    sign_in.click()
    email_field = driver.find_element_by_xpath(username_field)
    password_field = driver.find_element_by_xpath(password_field)
    email_field.send_keys(USERNAME)
    password_field.send_keys(PASSWORD + Keys.RETURN)


In [4]:
driver = browser_instance(executable_path, False, download_path)
# driver.get("https://cpj.org/data/killed/?status=Killed&motiveConfirmed%5B%5D=Confirmed&type%5B%5D=Journalist&cc_fips%5B%5D=BG&cc_fips%5B%5D=BT&cc_fips%5B%5D=IN&cc_fips%5B%5D=MV&cc_fips%5B%5D=NP&cc_fips%5B%5D=PK&cc_fips%5B%5D=CE&start_year=1992&end_year=2021&group_by=year")

In [5]:
def getter():
    counter = 0
    results = []
    driver.get("https://dbknews.com/category/news/")
    while counter < 9:
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/div/div[1]/div/nav/ul/li[8]/a')))
        table_finder = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[2]/div/div[1]/div/table/tbody')))
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        # print(soup.prettify())
        table = soup.find(class_="table")
        rows = table.find_all('tr')
        for row in rows[1:]:
          cells = row.find_all('td')
          results.append([cell.text for cell in cells])
    #       print(results)
        
        next_button.click()
        print('debug')
        counter = counter + 1
        print(counter)
#         return(results)


        with open('results.csv','w') as output_file:
            csvfile = csv.writer(output_file)
            csvfile.writerow(['name', 'organization','date', 'location', 'death','type','na'])
            csvfile.writerows(results)
getter()

debug
1
debug
2
debug
3
debug
4
debug
5
debug
6
debug
7
debug
8
debug
9


In [6]:
df = pd.read_csv('results.csv')
df


,name,organization,date,location,death,type,na
0,Abdul Aziz Shaheen,Azadi,"August 29, 2008",Pakistan,Killed,Crossfire,NaN
1,Abdul Hakim Shimul,Samakal,"February 3, 2017",Bangladesh,Killed,Dangerous Assignment,NaN
2,Abdul Haq Baloch,ARY Television,"September 29, 2012",Pakistan,Killed,Murder,NaN
3,Abdul Qadir Hajizai,WASH TV,"May 28, 2012",Pakistan,Killed,Murder,NaN
4,Abdul Razzak Johra,Royal TV,"November 3, 2008",Pakistan,Killed,Murder,NaN
...,...,...,...,...,...,...,...
175,Shahid Soomro,Kawish,"October 20, 2002",Pakistan,Killed,Murder,NaN
176,Shahid Zehri,Metro 1 News,"October 10, 2021",Pakistan,Killed,Murder,NaN
177,Shamsur Rahman,Janakantha,"July 16, 2000",Bangladesh,Killed,Murder,NaN
178,Shan Dahar,Abb Takk Television,"January 1, 2014",Pakistan,Killed,Murder,NaN


In [7]:
def get_names():
    names_list = []
    url_list = []
    base_url = "https://cpj.org/data/people/"
    desc_list = []
    for index, row in df.iterrows():
#         print(row["name"])
#         print('---')
        names_list.append(row["name"])
#     return(names_list)
    for name in names_list:
#             name = name.split(' ','')
        name = name.replace(' ', '-')
        name_w_url = base_url + name
#         print(name_w_url)
#         print('---')
        url_list.append(name_w_url)
    for link in url_list:
#         link = "https://cpj.org/data/people/ahmed-rilwan-abdulla/"
        url = link
        page = requests.get(url)
        soup = requests_get_item(url, "html")
        text = soup.find("article", class_="entry-content")
        desc_list.append(text.text)
    df['desc'] = desc_list
    print(df)
get_names()

                    name         organization                date    location  \
0     Abdul Aziz Shaheen                Azadi     August 29, 2008    Pakistan   
1     Abdul Hakim Shimul              Samakal    February 3, 2017  Bangladesh   
2       Abdul Haq Baloch       ARY Television  September 29, 2012    Pakistan   
3    Abdul Qadir Hajizai              WASH TV        May 28, 2012    Pakistan   
4     Abdul Razzak Johra             Royal TV    November 3, 2008    Pakistan   
..                   ...                  ...                 ...         ...   
175        Shahid Soomro               Kawish    October 20, 2002    Pakistan   
176         Shahid Zehri         Metro 1 News    October 10, 2021    Pakistan   
177       Shamsur Rahman           Janakantha       July 16, 2000  Bangladesh   
178           Shan Dahar  Abb Takk Television     January 1, 2014    Pakistan   
179     Shantanu Bhowmik              Dinraat  September 20, 2017       India   

      death                

In [8]:
df.to_csv("final.csv")

In [9]:
    for year in years:
        url = f"https://www.umpd.umd.edu/stats/arrest_report.cfm?year={year}"
        page = requests.get(url)
        soup = requests_get_item(url, "html")
        ls_rows = soup.find_all("tr")
        ls_rows_noHeaders = ls_rows[1:]
        num_rows = len(ls_rows_noHeaders)//2
        grouped = list(zip(*[iter(ls_rows_noHeaders)]*2))
        for i in grouped:
            subRow1 = i[0].find_all("td")
            subRow2 = i[1].find_all("td")

            arrest_number.append(subRow1[0].text.strip())
            arrested_date_time_charge.append(subRow1[1].text.strip())
            umpd_case_number.append(subRow1[2].text.strip())
            age.append(subRow1[3].text.strip())
            race.append(subRow1[4].text.strip())        
            sex.append(subRow1[5].text.strip())
            description.append(subRow2[0].text.strip())

NameError: name 'years' is not defined